In [1]:
# Paso 1.
# Modifica las variables de WRF para poder hacer el mapeo de cuantiles.

import numpy as np
import pandas as pd

import xarray as xr

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 5
c_lon = 5
client = Client( n_workers = 5, threads_per_worker = 5,
    memory_limit = "1.5GB" )
client

distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-pjovcb4z', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-uv9503gi', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-s83alejy', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-q2r6t_2o', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-p7ew7j5z', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 25,Total memory: 6.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50482,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 25
Started: Just now,Total memory: 6.98 GiB
Comm: tcp://127.0.0.1:50497,Total threads: 5
Dashboard: http://127.0.0.1:50501/status,Memory: 1.40 GiB
Nanny: tcp://127.0.0.1:50486,


In [2]:
# Cargamos el archivo.
n = 8
path_1 = f"../data/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
ds = xr.open_dataset( path_1, chunks = {
    "south_north": c_lat, "west_east": c_lon } )

# Creamos las variables.
ds["Wind Speed"] = np.sqrt( np.square(ds["U10"]) + np.square(ds["V10"]) )
ds["Wind Direction"] = np.arctan2( ds["V10"], ds["U10"] ) * 180 / np.pi - 90
ds["Wind Direction"] = ds["Wind Direction"].where(
    ds["Wind Direction"] > 0, ds["Wind Direction"] + 360 )
ds = ds.drop_vars( ["U10", "V10"] )
ds["Pvs"] = ( np.exp( 77.3450 + 0.0057*ds["T2"] - 7235/ds["T2"] )
    / ds["T2"]**8.2 )
ds["Relative Humidity"] = ( ds["Q2"] * ( ds["PSFC"]/ds["Pvs"] - ds["Pvs"] )
    / ( 10 * 286.9/461.4 ) )
ds["Relative Humidity"] = ds["Relative Humidity"].where(
    ds["Relative Humidity"] > 100, 100 )
ds = ds.drop_vars( ["Q2", "Pvs"] )

# Reordenamos el Dataset.
ds["Wind Speed"] = ds["Wind Speed"].assign_attrs( units = "m/s" )
ds["Relative Humidity"] = ds["Relative Humidity"].assign_attrs( units = "%" )
ds["Wind Direction"] = ds["Wind Direction"].assign_attrs( units = "degrees" )
ds = ds.rename_vars( { "XTIME": "time", "XLONG": "lon", "XLAT": "lat",
    "T2": "Temperature", "PSFC": "Pressure", "SWDOWN": "GHI" } )
ds["lon"] = ds["lon"].isel({"south_north": 0})
ds["lat"] = ds["lat"].isel({"west_east": 0})
ds = ds.swap_dims({"south_north": "lat", "west_east": "lon", "XTIME": "time"})

# Guardamos el archivo.
# Esta línea arranca el cómputo distribuido.
ds.to_netcdf( path_2 )

ds

<xarray.Dataset>
Dimensions:            (time: 262958, lon: 44, lat: 38)
Coordinates:
  * time               (time) datetime64[ns] 1985-01-01 ... 2014-12-31T18:00:00
  * lon                (lon) float32 -98.36 -98.33 -98.31 ... -97.36 -97.33
  * lat                (lat) float32 19.8 19.82 19.85 ... 20.58 20.61 20.63
Data variables:
    Temperature        (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
    Pressure           (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
    GHI                (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
    Wind Speed         (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
    Wind Direction     (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
    Relative Humidity  (time, lat, lon) float32 dask.array<chunksize=(262958, 5, 5), meta=np.ndarray>
Attributes: (12/141)
    CDI:                             Climate Data Interface version 2.1.0 (ht...
    Conventions:                     CF-1.6
    history:                         Tue Mar 12 08:39:57 2024: cdo distgrid,3...
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      1984-09-27_00:00:00
    SIMULATION_START_DATE:           1984-01-01_00:00:00
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    CDO:                             Climate Data Operators version 2.1.0 (ht...